### Imports

In [69]:
import os
import json
from openai.types.chat import ChatCompletionMessage, ChatCompletionMessageToolCall
from openai import AzureOpenAI
from tenacity import retry, wait_random_exponential, stop_after_attempt

### External Service Simulation
This could be replaced with any backend, api, integration service, etc...

In [53]:
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": unit})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

### Helper function to visualize responses from LLM about what function and parameters should be called

In [72]:
def pretty_print_chat_completion_message(chat_message: ChatCompletionMessage):
    content = chat_message.content
    print("Content:")
    print(json.dumps(content, indent=2))

    if chat_message.tool_calls:
        function_name = chat_message.tool_calls[0].function.name
        arguments = chat_message.tool_calls[0].function.arguments
        print("Function Name:", function_name)
        print("Arguments:")
        print(json.dumps(json.loads(arguments), indent=2))
    else:
        print("Functions: None")

### Azure OpenAI client setup

In [55]:
deployment = "ai-roadshow" #gpt-3.5-turbo-0613
client = AzureOpenAI(
    api_key=os.getenv("AI_ROADSHOW_AOAI_KEY"),  
    api_version="2023-12-01-preview",
    azure_endpoint=os.getenv("AI_ROADSHOW_AOIA_ENDPOINT")
)

### Model interaction method with retry policy

In [56]:
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, tools=None, tool_choice=None, model=deployment):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

### List of functions available for our aplication

In [57]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city for which to get the weather",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["fahrenheit", "celsius"],
                        "description": "The temperature unit to use. Default is fahrenheit.",
                    },
                },
                "required": ["location", "format"],
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_n_day_weather_forecast",
            "description": "Get an N-day weather forecast",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The location the user wants to know its weather",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Default is fahrenheit.",
                    },
                    "num_days": {
                        "type": "integer",
                        "description": "The number of days to forecast",
                    }
                },
                "required": ["location", "format", "num_days"]
            },
        }
    },
]

### Testing specifying location

In [73]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "I'm visiting New York. What will be the weather today?"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
pretty_print_chat_completion_message(assistant_message)




Content:
null
Function Name: get_current_weather
Arguments:
{
  "location": "New York",
  "format": "fahrenheit"
}


### Testing specifying the location and the preferred units

In [75]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Please tell me the weather in Bogota, in celcius"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
pretty_print_chat_completion_message(assistant_message)

Content:
null
Function Name: get_current_weather
Arguments:
{
  "location": "Bogota",
  "format": "celsius"
}


### Testing without specifying location

In [82]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "How's going to be the weather?"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
pretty_print_chat_completion_message(assistant_message)

Content:
"Sure, could you please provide me with the location for which you would like to know the weather?"
Functions: None


#### User answers the ask for location specification

In [83]:
messages.append({"role": "user", "content": "I am in Paris"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
assistant_message
pretty_print_chat_completion_message(assistant_message)


Content:
null
Function Name: get_current_weather
Arguments:
{
  "location": "Paris",
  "format": "celsius"
}


### Testing the other function

In [84]:
import json
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "How will be the weather in Cartagena for the next week?"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
pretty_print_chat_completion_message(assistant_message)


Content:
null
Function Name: get_n_day_weather_forecast
Arguments:
{
  "location": "Cartagena",
  "format": "fahrenheit",
  "num_days": 7
}
